In [438]:
import pandas as pd
import re
import numpy as np

In [1290]:
clubs = pd.read_csv('club_info.csv', index_col=0)
rankings = pd.read_csv('ranking_data.csv', index_col=0)

In [1291]:
pd.set_option('display.max_rows', None)

In [1310]:
clubs

,club_name,club_loc,club_id,club_zone,og_loc,pool_name,address,state,city,zip,loc
0,209 Water Polo Club,"1800 Buhach RoadAtwater, CA",31318,10008,Primary Pool:Buhach Colony High School1800 Buh...,Buhach Colony High School,1800 Buhach Road,CA,Atwater,NaN,NaN
1,A-Town Water Polo Club,"1 High School HillAtascadero, CA93422",30234,10008,Primary Pool:Atascadero High School1 High Scho...,Atascadero High School,1 High School Hill,CA,Atascadero,93422,"1 High School Hill, Atascadero, CA 93422"
2,Alumni Water Polo Club,"1 Grizzly WayGranite Bay, CA95746",23883,10008,Primary Pool:Granite Bay High School Pool1 Gri...,Granite Bay High School Pool,1 Grizzly Way,CA,Granite Bay,95746,"1 Grizzly Way, Granite Bay, CA 95746"
3,American River Water Polo Club,"4540 American River DrSacramento, CA95864",85,10008,Primary Pool:Rio Americano High School4540 Ame...,Rio Americano High School,4540 American River Dr,CA,Sacramento,95864,"4540 American River Dr, Sacramento, CA 95864"
4,Arroyo Grande Water Polo,"495 Valley RdArroyo Grande, CA93420",21063,10008,Primary Pool:Arroyo Grande High School495 Vall...,Arroyo Grande High School,495 Valley Rd,CA,Arroyo Grande,93420,"495 Valley Rd, Arroyo Grande, CA 93420"
5,Asa Aquatics,"100 FarmlandMerced, CA95348",19513,10008,Primary Pool:El Capitan High School (Merced)10...,El Capitan High School,100 Farmland,CA,Merced,95348,"100 Farmland, Merced, CA 95348"
6,Bakersfield Water Polo Club,"2800 Loma Linda DrBakersfield, CA93312",16629,10008,Primary Pool:Garces Memorial High School2800 L...,Garces Memorial High School,2800 Loma Linda Dr,CA,Bakersfield,93312,"2800 Loma Linda Dr, Bakersfield, CA 93312"
7,BarDown Water Polo Academy,"One University CircleTurlock, CA95382",31674,10008,Primary Pool:CSU StanislausOne University Circ...,CSU Stanislaus,One University Circle,CA,Turlock,95382,"One University Circle, Turlock, CA 95382"
8,Blazer Water Polo Club,"1717 N McauliffVisalia, CA93292",16120,10008,Primary Pool:Golden West High School1717 N. Mc...,Golden West High School,1717 N Mcauliff,CA,Visalia,93292,"1717 N Mcauliff, Visalia, CA 93292"
9,Bulldog Water Polo Club,"5305 N Campus DrFresno, CA93740",31612,10008,Primary Pool:Fresno State Aquatics Complex5305...,Fresno State Aquatics Complex,5305 N Campus Dr,CA,Fresno,93740,"5305 N Campus Dr, Fresno, CA 93740"


In [1293]:
rankings.head()

,Finish,Team,Age,Sex,Year,og_name,Squad,Club,Tie
0,1,LAGUNA BEACH WATER POLO,10,Girls,2016,LAGUNA BEACH WATER POLO,A,LAGUNA BEACH,0.0
1,1,VANGUARD AQUATIC,12,Boys,2016,VANGUARD AQUATICS,A,VANGUARD,0.0
2,1,LO ANGELE PREMIER,14,Boys,2016,LOS ANGELES PREMIER,A,LO ANGELE PREMIER,0.0
3,1,STANFORD WATER POLO FOUNDATION,16,Boys,2016,STANFORD WATER POLO FOUNDATION,A,STANFORD,0.0
4,1,REGENCY WATER POLO CLUB,18,Boys,2016,REGENCY WATER POLO CLUB,A,REGENCY,0.0


## Cleaning club data

In [1294]:
clubs['club_id'] = clubs['club_id'].str.replace('Club ID:','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\r','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\n','').str.strip()
clubs['club_id'] = clubs['club_id'].str.replace('\t','').str.strip()
clubs.replace(u'\xa0',u'', regex=True, inplace=True)
clubs.replace(u'\r',u'', regex=True, inplace=True)
clubs.replace(u'\n',u'', regex=True, inplace=True)

In [1295]:
clubs['og_loc'] = clubs['club_loc']
clubs['club_loc'] = clubs['club_loc'].str.replace('Primary Pool:', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('23Rd', '23rd', regex=True)
clubs['club_loc'] = clubs['club_loc'].str.replace('90Th', '90th')
clubs['club_loc'] = clubs['club_loc'].str.replace('18Th', '18th')
clubs['club_loc'] = clubs['club_loc'].str.replace('10Th', '10th')
clubs['club_loc'] = clubs['club_loc'].str.replace('6Th', '6th')
clubs['club_loc'] = clubs['club_loc'].str.replace('136Th', '136th')
clubs['club_loc'] = clubs['club_loc'].str.replace('71St', '71st')
clubs['club_loc'] = clubs['club_loc'].str.replace('M/S NG27', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('.', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('\([\w\s]+\)', '')
clubs['club_loc'] = clubs['club_loc'].str.replace('Dr, ', 'Dr')
clubs['club_loc'] = clubs['club_loc'].str.replace('DR', 'Dr')
clubs['club_loc'] = clubs['club_loc'].str.replace('RD', 'Rd')
clubs['club_loc'] = clubs['club_loc'].str.replace('&', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('-', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('/', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace(':', ' ')
clubs['club_loc'] = clubs['club_loc'].str.replace('\'', '')

clubs['club_loc'] = clubs['club_loc'].str.replace('[\w\s]+[(?:\\t)]+', '', 1)


In [1296]:
clubs['pool_name'] = clubs['club_loc'].str.extract('(?<=\\t)([^(?:\\t)]+)(?=\\t)')
clubs['club_loc'] = clubs['club_loc'].str.replace('[\w\s]+[(?:\t)]+', '', 1)

In [1297]:
clubs[clubs['pool_name'].isnull()]

,club_name,club_loc,club_id,club_zone,og_loc,pool_name
131,Calgary Masters,Unknown,19401,17972,Unknown,NaN
132,Calgary Polo Splash,Unknown,17376,17972,Unknown,NaN
212,Barcelona International Water Polo Association,Unknown,34590,1795,Unknown,NaN
394,Soda Moms,Unknown,34704,10006,Unknown,NaN


In [1298]:
clubs['club_loc'] = clubs['club_loc'].str.replace('Rd ', 'Rd')
clubs['club_loc'] = clubs['club_loc'].str.replace('Blvd ', 'Blvd')
clubs['club_loc'] = clubs['club_loc'].str.replace('Lincoln ', 'Lincoln')
clubs['club_loc'] = clubs['club_loc'].str.replace('St, ', 'St')
clubs['club_loc'] = clubs['club_loc'].str.replace('St ', 'St')
clubs.loc[clubs['pool_name'].str.contains(',', na=False), 'club_loc'] = clubs['club_loc'].str.replace(',',' ',1).str.strip()

EDITING - doesn't work with multiple capital letters
clubs['address'] = clubs['club_loc'].str.extract('([\w\s-]+),')
clubs['address'] = clubs['address'].str.extract('(.+?)(?=\B[A-Z,])')

In [1299]:
clubs['address'] = clubs['club_loc'].str.extract('([\w\s-]+),')

In [1300]:
clubs.loc[clubs['address'].isnull(), 'club_loc'] = clubs['club_loc'].str.replace(',',' ',1).str.strip()
clubs.loc[clubs['address'].isnull(), 'club_loc'] = clubs['club_loc'].str.replace('[\w\s]+[(?:\t)]+', '', 1)
clubs.loc[clubs['address'].isnull(), 'address'] = clubs['club_loc'].str.extract('([\w\s]+),')
clubs.loc[clubs['address'].isnull(), 'address'] = clubs['address'].str.extract('(.+)(?=\B[A-Z,])')

In [1307]:
clubs.iloc[355,9]

'94550'

In [1312]:
clubs[clubs['address'].isnull()]

(29, 11)

In [1303]:
clubs['state'] = clubs['club_loc'].str.extract('([A-Z]{2})')

clubs['city'] = clubs['club_loc'].str.extract('[a-z]([A-Z][a-z]+\s*\w*)')

In [1304]:
clubs.replace(u'\t',u'', regex=True, inplace=True)
clubs.replace(u'\r',u'', regex=True, inplace=True)
clubs.replace(u'\n',u'', regex=True, inplace=True)

In [1305]:
clubs['zip'] = np.nan
clubs['zip'] = clubs['club_loc'].str.extract('([0-9]{5})\s?[0-9]*$')

clubs = clubs.drop(['club_loc', 'pool_loc'], axis=1)

In [1306]:
clubs['loc'] = clubs['address'] + ", " + clubs['city'] + ", " + clubs['state'] + " " + clubs['zip']

In [250]:
clubs.shape

(540, 11)